# VAE (Variational Auto Encoder)

## I. 모델 이해

#### 목표: Generative Model
* 데이터 X의 잠재변수 z(Latent Vector)를 도출하고 
* z의 random sample 을 통해 데이터 X와 유사한 결과를 생성

<img src="../../shared/VAE_intuition.png" alt="Drawing" style="width: 500px;" align="left"/>

#### 구성: Encoder + Decoder

* **Encoder**: Recognition Model 의 역할을 한다고 해석할 수 있음
* **Decoder**: Generative Model 의 역할을 한다고 해석할 수 있고, "실제 VAE의 목표"가 되는 부분

<img src="../../shared/VAE_abstract_start.png" alt="Drawing" style="width: 500px;" align="left"/>

#### 과정:

<img src="../../shared/VAE_step.png" alt="Drawing" style="width: 500px;" align="left"/>

(1) **Variational Inference** - 데이터 X로부터의 잠재변수 z를 decode했을 때 "실제와 유사한" 데이터를 잘 생성하는 z의 분포($p(z|x)$) 를 찾는 목표의 대안으로.. 
* $ p(z|x) = {p(x|z)p(z)\over{p(x)}}$ 를 찾기 위해 실제 "모든" 데이터 X의 분포 $ P(x) $를 알아야하는데 이는 train 데이터만으로 알 수 없음 ($ \; p(x) = \int{p(x|z)p(z)dz} \;$ 계산 불가능)
* 따라서 어떤 "계산 가능한" 확률분포 $ q(z|x) $를 두어 $ q(z|x) $ 가 $ p(z|x) $에 근사하도록 학습 (Variational Inference)

(2) **argmax$ELBO(\phi)$** - $ q(z|x)$의 모수(parameter) $ \phi $를 조정하여 min$KL(q(z|x)||p(z|x))$ 가 되게 하는 모수를 찾는 목표의 대안으로..
* $p(x)$를 알기 위해 $log(p(x))$를 계산해보면 $log(p(x)) = ELBO(\phi) + KL(q(z|x)||p(z|x))$ 형태로 표현됨
* KL-divergence 를 최소화하는 $q(z|x)$의 모수 $\phi$ 를 찾으면 되는데 $(p(z|x))$를 모르기 때문에 KL-term을 최소화하는 대신 ELBO-term을 최대화하는 $\phi$를 찾는 것이 목표

(3) **Reparameterization Trick** - ELBO-term을 극대화하는 학습을 할 때 Backpropagation을 가능하게 하기 위해
* feed forward과정 내에 있는 $q(z|x)$분포로부터의 z를 sampling 하는것은 미분이 가능한 연산이 아니므로 BP 불가
* 따라서 $ z = \mu + logVar $ (non-deterministic하므로)가 아닌 $ z = \mu + \epsilon*(logVar) $ 로 변형 (eps: $N(0,1)$로 부터의 random sampling)
* ELBO-term $= E_{q(z|x)}[log(p(x|z))] - KL(q_{\phi}(z|x_i)||p(z))$ 에서 

(4) **Maximum Likelihood Estimation** - Decoder를 통해 잠재변수 z로부터 X와 근사한 분포를 추정(Reconstruct)하고 Encoder를 통해 X로부터 추출한 z가 사전분포 p(z)와 근사하도록 추정(Regularization)

* **Reconstruction Error**(ELBO의 첫번째 term): $g_\theta(z)$ 와 데이터 $X$의 분포를 최대로 유사하는 방향으로 학습하여 $ E_{q(z|x)}[log(p(x|z))]$ 를 최대화
    * 방법1: ** $g_\theta(z)$의 결과를 Bernnoulli Distribution 으로 가정**하여 $p_\theta(x_i|z^i)$를 도출하고, 수식($log(p_\theta(x_i|z^i)$)을 정리하면 $p_{i,j}$와 $X_{i,j}$의 **Cross Entropy 형태**
    * 방법2: ** $g_\theta(z)$의 결과를 Gaussian Distribution 으로 가정**하여 $\mu, \sigma$를 도출하고, 수식($log(p_\theta(x_i|z^i)$)을 정리하면 **$\mu_{i,j}$와 $X_{i,j}$의 MSE 형태**
* **Regularization Error**(ELBO의 두번째 term): $q_{\phi}(z|x_i)$를 "아는 사전 분포" $z$~$N(0,1)$와 유사한 방향으로 학습하여 $KL(q_{\phi}(z|x_i)||p(z))$ 를 최소화

<img src="../../shared/VAE_loss.png" alt="Drawing" style="width: 500px;" align="left"/>

<img src="../../shared/VAE_loss-all.png" alt="Drawing"/>

## II. MNIST 를 통한 예시

#### (0) Define Hyper-parameters / Helper Function

In [3]:
import torch
import os

In [4]:
# Device Configuration for Where the Tensors Be Operated
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define OS Configuration
sample_dir = './results'

# Hyper-parameters
image_size = 784
h_dim = 400
z_dim = 20

num_epochs = 20
batch_size = 128
learning_rate = 1e-3

#### (1) Load Data

#### (2) Define Dataloader

#### (3) Define Model

#### (4) Set Loss & Optimizer

#### (5) Train / Test

## cf) Process in Visualization

<hr>

## reference

* [오토인코더의 모든 것](https://www.youtube.com/watch?v=o_peo6U7IRM)<br>
* [https://www.jeremyjordan.me/variational-autoencoders/](https://www.jeremyjordan.me/variational-autoencoders/)<br>
* [https://towardsdatascience.com/understanding-variational-autoencoders-vaes-f70510919f73](https://towardsdatascience.com/understanding-variational-autoencoders-vaes-f70510919f73)